In [38]:
import pandas as pd
import seaborn as sns

In [39]:
train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

In [40]:
log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [41]:
log_train_df= log_train
train_df = train

In [42]:
14*60

840

In [43]:
train_df.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X131,X132,X133,X134,X135,X136,X137,X138,X139,label
0,7054.0,2.015152,0.0,32.0,NaN,4.0,NaN,NaN,7.0,6.0,...,0.0,NaN,NaN,1.0,1.0,1.0,3.0,2.0,NaN,1
1,147131.0,1.000000,NaN,2.0,NaN,NaN,0.0,NaN,6.0,5.0,...,NaN,NaN,0.0,3.0,1.0,1.0,2.0,10.0,NaN,1
2,132068.0,1.000000,NaN,1.0,NaN,2.0,NaN,0.0,9.0,1.0,...,NaN,NaN,0.0,1.0,1.0,2.0,2.0,4.0,NaN,1
3,85195.0,2.071429,0.0,1.0,NaN,2.0,NaN,NaN,8.0,7.0,...,NaN,NaN,0.0,2.0,2.0,1.0,4.0,2.0,NaN,1
4,191948.0,2.000000,NaN,2.0,0.0,2.0,NaN,NaN,14.0,2.0,...,0.0,0.0,NaN,2.0,2.0,1.0,2.0,6.0,0.0,1


In [52]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')



train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [53]:
#train_df_enriched.head()
train_df_enriched.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X134,X135,X136,X137,X138,X139,label,event_count,unique_sources,avg_time_diff
0,7054.0,2.015152,0.0,32.0,NaN,4.0,NaN,NaN,7.0,6.0,...,1.0,1.0,1.0,3.0,2.0,NaN,1,0.0,0.0,0.000000
1,147131.0,1.000000,NaN,2.0,NaN,NaN,0.0,NaN,6.0,5.0,...,3.0,1.0,1.0,2.0,10.0,NaN,1,1.0,1.0,0.000000
2,132068.0,1.000000,NaN,1.0,NaN,2.0,NaN,0.0,9.0,1.0,...,1.0,1.0,2.0,2.0,4.0,NaN,1,0.0,0.0,0.000000
3,85195.0,2.071429,0.0,1.0,NaN,2.0,NaN,NaN,8.0,7.0,...,2.0,2.0,1.0,4.0,2.0,NaN,1,40.0,2.0,1182.717949
4,191948.0,2.000000,NaN,2.0,0.0,2.0,NaN,NaN,14.0,2.0,...,2.0,2.0,1.0,2.0,6.0,0.0,1,0.0,0.0,0.000000


In [ ]:
train_df_enriched.shape

(84377, 144)

In [70]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt

# The rest of your code for loading and cleaning the data remains the same.
train_clean = train_df_enriched.dropna(subset=['label'])

# Separate features and target variable after dropping NaN in target
#X = train_clean.drop(['label', 'ID'], axis=1) #features

#top20_f= feature_importances.nlargest(20).index

X = train_clean.drop(['label','ID'],axis=1)
y = train_clean['label']              #target

# Initialize the SimpleImputer with 'mean' strategy
imputer = SimpleImputer(strategy='median')

# Fit the imputer and transform the dataset
X_imputed = imputer.fit_transform(X)


# Convert the imputed data back into a DataFrame (useful for further operations)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

#ADASYN
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=42)

# Initialize the Gradient Boosting classifier
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model on your training data
gbc.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = gbc.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy score
print(f"Accuracy of the Gradient Boosting classifier: {accuracy:.2f}")
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy of the Gradient Boosting classifier: 0.92
[[6055  722]
 [ 369 6366]]
0.9192569567791593
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      6777
           1       0.90      0.95      0.92      6735

    accuracy                           0.92     13512
   macro avg       0.92      0.92      0.92     13512
weighted avg       0.92      0.92      0.92     13512



In [71]:
from sklearn.metrics import accuracy_score,f1_score
print('F1_Score :',f1_score(y_test,y_pred))
#F1_Score : 0.9221600231637873 
#F1_Score : 0.9229871645274212 - 0.1 median

F1_Score : 0.9210735730304564


In [66]:
log_test_df=log_test
test_df = test

In [67]:
log_test_df['time'] = pd.to_datetime(log_test_df['time'])

# Extract features
log_test_df['hour'] = log_test_df['time'].dt.hour
log_test_df['day_of_week'] = log_test_df['time'].dt.dayofweek
log_test_df['day'] = log_test_df['time'].dt.day

# Count of events per ID
event_count = log_test_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_test_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
test_df_enriched = test_df.merge(event_count, on='ID', how='left')
test_df_enriched = test_df_enriched.merge(unique_sources, on='ID', how='left')



# Time difference features (e.g., average time between events for each ID)
log_test_df['time_diff'] = log_test_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_test_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')



test_df_enriched = test_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
test_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, 'avg_time_diff':0}, inplace=True)

In [68]:
test_df_enriched.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X133,X134,X135,X136,X137,X138,X139,event_count,unique_sources,avg_time_diff
0,17547.0,1.272727,0.0,1.0,0.0,3.0,0.0,0.0,7.0,26.0,...,0.0,2.0,6.0,2.0,4.0,3.0,NaN,2.0,2.0,124.00
1,140449.0,1.000000,0.0,1.0,NaN,1.0,NaN,0.0,8.0,4.0,...,0.0,3.0,2.0,2.0,2.0,5.0,NaN,4.0,1.0,37.00
2,182658.0,2.000000,NaN,2.0,NaN,2.0,NaN,0.0,64.0,2.0,...,NaN,2.0,4.0,1.0,1.0,7.0,0.0,2.0,2.0,155.00
3,149652.0,1.000000,0.0,1.0,NaN,1.0,0.0,0.0,9.0,9.0,...,NaN,3.0,1.0,1.0,2.0,7.0,NaN,5.0,1.0,95.25
4,106304.0,2.000000,NaN,2.0,NaN,3.0,NaN,0.0,1.0,6.0,...,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.00


In [69]:
# Convert the 'ID' column from float to int
test_df_enriched['ID'] = test_df_enriched['ID'].astype(int)

# Separate out the 'ID' column and features from the 'test' dataset
test_features = test_df_enriched.drop('ID', axis=1)

# Impute the 'test' dataset using the fitted SimpleImputer
# Note: We use .transform() here, NOT .fit_transform()
test_imputed = imputer.transform(test_features)

# Predict the labels using the trained and tuned Random Forest model (best_grid)
predicted_labels = gbc.predict(test_imputed)

# Create a DataFrame with 'ID' and the predicted 'label'
results_df = pd.DataFrame({
    'ID': test_df_enriched['ID'],
    'LABEL': predicted_labels
})

# Convert the 'ID' column to integer to match the original ID's format
results_df['ID'] = results_df['ID'].astype(int)

# Output the DataFrame with 'ID' and 'label'
# If you want to save to a CSV file:
results_df.to_csv('/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Submissions/my_submission_17_gbc_ADASYN_log_10.csv', index=False)

# If you want to display the first few rows of the DataFrame
print(results_df.head())

       ID  LABEL
0   17547      0
1  140449      1
2  182658      1
3  149652      1
4  106304      1


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print(results_df.shape)

(35982, 2)
